In [14]:
import chromadb
import requests
import json
import os
def petsmart_query(user_query):
    # ChromaDB query
    client = chromadb.CloudClient(
        api_key='ck-GbrLBVwv1NppMUTvq38k5MwMxinBaRtaa6oyAiAFxHDN',
        tenant='986f4b43-03a2-4f26-a81f-311f83dff543',
        database='petsmart'
    )
    collection = client.get_or_create_collection(name="petsmart_documents")
    chroma_results = collection.query(
        query_texts=[user_query],
        n_results=10
    )

    # Langflow API call
    api_key = "sk-wYgAQ3w9IZlhOBBhrzuuqLpyfmVWLSZk2QHOMlVYZgo"
    os.environ["LANGFLOW_API_KEY"] = api_key
    url = "https://langflows.vercel.app/api/v1/run/957834e3-c519-40ea-a494-4f732a882d28"

    payload = {
        "input_value": user_query,
        "output_type": "chat",
        "input_type": "chat"
    }

    headers = {
        "Content-Type": "application/json",
        "x-api-key": api_key
    }

    response = requests.post(url, json=payload, headers=headers)
    data = json.loads(response.text)
    api_answer = data["outputs"][0]["outputs"][0]["artifacts"]["message"]

    # Build chunks
    chunks = []
    for i in range(len(chroma_results['documents'][0])):
        chunk = {
            'text': chroma_results['documents'][0][i],
            'metadata': {
                'page': chroma_results['metadatas'][0][i].get('page', 'N/A'),
                'document': chroma_results['metadatas'][0][i].get('document', 'Unknown'),
                'snippet': chroma_results['metadatas'][0][i].get('snippet', '')[:50] + '...',
                'pdf_path': chroma_results['metadatas'][0][i].get('pdf_path', ''),
                'links_count': chroma_results['metadatas'][0][i].get('links_count', 0),
                'links_json': chroma_results['metadatas'][0][i].get('links_json', '[]')
            },
            'similarity': round(1 - (chroma_results['distances'][0][i] / 2), 3)
        }
        chunks.append(chunk)

    # Merged response
    result = {
        'answer': api_answer,
        'chunks': chunks
    }

    return result

In [19]:
response = petsmart_query("do you remember what i just asked?")

In [20]:
response

{'answer': 'Yes, you asked how to become a partner with PetSmart Charities. To become a partner, you can start by visiting their "Partner with Us" page, which provides resources and information on collaboration, including access to logos and marketing support. Here are the links:\n\n- [Partner with Us (US)](https://petsmartcharities.org/pro/resources/marketing-support)\n\n- [Partner with Us (Canada)](https://petsmartcharities.ca/pro/resources/marketing-support)\n\nAdditionally, you can explore the [Adoption Partner page](https://petsmartcharities.org/pro/adoption-partner) for more details on adoption partnerships. If you have further questions or need assistance, you can reach out to the PetSmart Charities Adoptions team via email at [Adoptions@PetSmartCharities.org](mailto:Adoptions@PetSmartCharities.org).',
 'chunks': [{'text': '31\nPetSmart Charities\nADOPTION PARTNER MANUAL - US/PR    I    2025-26\nTHANK YOU\nany questions?\nFor questions regarding information provided in this manu

In [18]:
print(response["answer"])

To become a partner with PetSmart Charities, you can start by visiting their "Partner with Us" page. This page provides resources and information on how to collaborate with PetSmart Charities, including access to logos and marketing support. You can find this page at the following links:

- [Partner with Us (US)](https://petsmartcharities.org/pro/resources/marketing-support)

- [Partner with Us (Canada)](https://petsmartcharities.ca/pro/resources/marketing-support)

Additionally, you can explore the [Adoption Partner page](https://petsmartcharities.org/pro/adoption-partner) for more details on adoption partnerships. If you have further questions or need assistance, you can reach out to the PetSmart Charities Adoptions team via email at [Adoptions@PetSmartCharities.org](mailto:Adoptions@PetSmartCharities.org).
